In [12]:
import json
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
import pickle 

In [13]:
data_file = open("yelp_academic_dataset_review.json")

In [14]:
positive_sentence = 0
sentence_count = 0
maximum_sentence_length = 0
word_count = dict()
dataset_size = 1000000

In [15]:
for line in data_file:
    entry = json.loads(line)
    review = entry['text']
    is_positive = entry['stars'] >= 3.0
    if is_positive:
        positive_sentence += 1

    sentences = sent_tokenize(review)
    if len(sentences) > 10:
        continue

    if sentence_count >= dataset_size:
        break

    for sentence in sentences:

        words = word_tokenize(sentence.lower())
        words_length = len(words)
        if words_length > 15:
            continue
        sentence_count += 1

        maximum_sentence_length = max(maximum_sentence_length, words_length)

        for word in words:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

In [16]:
vocab = dict()
vocab['<unk>'] = 0
vocab[u'\u25A1'] = 1
next_index = 2

for word, count in word_count.items():
    if count <= 5:
        continue
    vocab[word] = next_index
    next_index += 1

In [17]:
dataset = []
ratings = []
sentence_count = 0

for line in data_file:
    entry = json.loads(line)
    review = entry['text']
    is_positive = entry['stars'] >= 3.0

    sentences = sent_tokenize(review)
    if len(sentences) > 10:
        continue
    
    if sentence_count >= dataset_size:
        break

    for sentence in sentences:
        indexes = []
        words = word_tokenize(sentence.lower())
        words_length = len(words)
        if words_length > 15:
            continue
        sentence_count += 1

        for word in words:
            if word in vocab:
                indexes.append(vocab[word])
            else:
                indexes.append(0)
        while len(indexes) < maximum_sentence_length+1:
            indexes.append(1)

        dataset.append(indexes)
        ratings.append(is_positive)

In [18]:
np.savetxt("sentiment_dataset_reviews.txt", dataset, fmt="%d")
np.savetxt("positive_ratings.txt", ratings,  fmt="%d")

In [19]:
with open('initial_vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [20]:
len(vocab)

17912